In [1]:
import pandas as pd
from typing import Literal
import paths

folder = paths.ObsidianFolder


####################
# Obsidian CANVAS
####################
def SetNode(id: str, x: int, y: int, color=0, nome: str = ""):
    if len(nome) < 25:
        height = 220
    elif len(nome) < 50:
        height = 260
    else:
        height = 300
    return f'{{"id":"{id}","type":"file","file":"PlanoMaterias/Subjects/{id}.md","width":440,"height":{height},"color":"{color}","x":{x},"y":{y}}}'


def SetEdge(
    id: str,
    fromNode: str,
    fromSide: Literal["top", "bottom", "left", "right"],
    toNode: str,
    toSide: Literal["top", "bottom", "left", "right"],
    color=0,
):
    return f'{{"id":"{id}","fromNode":"{fromNode}","fromSide":"{fromSide}","toNode":"{toNode}","toSide":"{toSide}","toEnd":"none","color":"{color}"}}'


CANVAS_Y = [0] * 11
Nodes = pd.DataFrame(index=["color", "x", "y", "nome"])
Edges = pd.DataFrame(index=["fromNode", "fromSide", "toNode", "toSide", "color"])


def SubjectToCANVAS(subjectDF):
    global CANVAS_Y
    CODIGO, NOME, *_ = subjectDF.name.split(" - ")
    PERIODO = int(subjectDF["Período"])
    TIPO = "".join(subjectDF["Tipo"]).replace("'", "")
    preRequisitos = [i.replace("'", "") for i in subjectDF["Pré-Requisitos"]]
    coRequisitos = [i.replace("'", "") for i in subjectDF["Co-Requisitos"]]
    equivalencias = [i.replace("'", "") for i in subjectDF["Equivalências"]]
    if CODIGO in aprovadas:
        color = 0
    elif TIPO == "OPTATIVO":
        color = 6
    else:
        color = PERIODO % 2 + 4
    Nodes[CODIGO] = [
        color,
        600 * PERIODO,
        CANVAS_Y[PERIODO],
        NOME,
    ]
    CANVAS_Y[PERIODO] += 500

    for preReq in preRequisitos:
        Edges[f"{CODIGO}-{preReq}"] = [
            CODIGO,
            "left",
            preReq,
            "right",
            0 if preReq in aprovadas else 1,
        ]


####################
# PerfilCurricular to MD
####################
def FormatSubjectData(subjectData):
    if isinstance(subjectData, list):
        if not subjectData:
            return ""
        else:
            return str("[[" + "]], [[".join(subjectData) + "]]").replace("'", "")
    return subjectData


def SubjectToMD(subjectDF):
    global SumPeriodos
    CODIGO, NOME, *_ = subjectDF.name.split(" - ")
    periodo = subjectDF["Período"]
    conteudo = f"""### {NOME}
|**Período**|**Tipo**|**CH Total**|
|-|-|-|
| {periodo} | {''.join(subjectDF['Tipo']).replace("'", "")} | {subjectDF['CH Total']} |
##### Pré-Requisitos
{FormatSubjectData(subjectDF['Pré-Requisitos'])}
##### Có-Requisitos
{FormatSubjectData(subjectDF['Co-Requisitos'])}
##### Equivalências
{FormatSubjectData(subjectDF['Equivalências'])}
##### Ementa
{''.join(subjectDF['Ementa']).replace("'", "")}
"""
    with open(f"{folder}Subjects\\{CODIGO}.md", "w", encoding="utf-8") as f:
        f.write(conteudo)


####################
# Main
####################
# Histórico
historico = pd.read_excel("Historico.xlsx", index_col=0)
aprovadas = [
    i.split(" - ")[0]
    for i in historico.loc[
        historico["Situação"].isin(["APROVADO POR MÉDIA", "DISPENSADO"])
    ].index
]
# Perfil
PerfilCurricular = pd.read_excel("PRO03.xlsx", index_col=0).T
PerfilCurricular = PerfilCurricular.applymap(
    lambda x: (
        [i for i in x.strip("[]").replace("", "").split(", ") if i != ""]
        if isinstance(x, str)
        else x
    )
)

# Subject to MD
for subject in PerfilCurricular:
    SubjectToMD(PerfilCurricular[subject])
    SubjectToCANVAS(PerfilCurricular[subject])

# Set Color by Needed
preReqSum = pd.Series([Edges[edge]["toNode"] for edge in Edges]).value_counts()
for subject in preReqSum.keys():
    selectedSubjects = [
        Edges[edge].name for edge in Edges if Edges[edge]["toNode"] == subject
    ]
    for edge in selectedSubjects:
        if Edges[edge]["color"] != 0:
            if preReqSum[subject] > 2:
                Edges[edge]["color"] = 1
            elif preReqSum[subject] > 1:
                Edges[edge]["color"] = 2
            else:
                Edges[edge]["color"] = 4

In [19]:
import networkx as nx

# Cria um grafo direcionado
G = nx.DiGraph()
for node in Nodes:
    G.add_node(node, x=Nodes[node])
for edge in Edges:
    G.add_edge(Edges[edge]["fromNode"], Edges[edge]["toNode"])

# Classifica os nós com base no número de arestas conectadas
nodes_sorted = sorted(G.nodes(), key=G.degree, reverse=True)

# Atribui a posição Y com base na classificação e mantém a posição X fixa
pos = {node: (G.nodes[node]["x"], i * 500) for i, node in enumerate(nodes_sorted)}

# Atualiza a posição Y no DataFrame
for node, position in pos.items():
    Nodes[node]["y"] = position[1]